In [2]:
from hvec_importers.rws import communicators as com
from hvec_importers.rws import rws
from hvec_importers.rws import helpers as hlp
from hvec_importers.rws import parsers as parse
import dateutil
import pandas as pd
import requests
import warnings
import swifter

In [3]:
com.TIMEOUT

90

Understanding the raw data structure of data imported from RWS Waterinfo. Goal is to optimise the parsing procedure. The one in ddlpy relies on nested for-loops and is therefore very slow.

Get some data first. A deep dive in the library is necessary to obtain some raw data to experiment on. Do not try this at home!

In [4]:
warnings.simplefilter('ignore')

In [5]:
locations = rws.station_list()

In [24]:
sel = hlp.create_selection_table(
    locations = locations,
    name = 'Keeten boei', quantity = 'Hm0')

In [25]:
sel

,Code,Locatie_MessageID,Coordinatenstelsel,X,Y,Naam,Parameter_Wat_Omschrijving,Compartiment.Code,Compartiment.Omschrijving,Eenheid.Code,Eenheid.Omschrijving,Grootheid.Code,Grootheid.Omschrijving,Hoedanigheid.Code,Hoedanigheid.Omschrijving,Parameter.Code,Parameter.Omschrijving,start,end
0,KEET,18022,25831,566845.068456,5.717765e+06,Keeten boei,Significante golfhoogte in het spectrale domei...,OW,Oppervlaktewater,cm,centimeter,Hm0,Significante golfhoogte in het spectrale domein,F003-050,golffrequentie tussen 30 en 500 mHz,NVT,Waarde is niet van toepassing,1800-01-01,2100-12-31


In [26]:
code = 'KEET'
sel = sel.query('Code == @code')

In [28]:
start = dateutil.parser.parse('1680-01-01')
end = dateutil.parser.parse('2022-1-3')

In [29]:
sel

,Code,Locatie_MessageID,Coordinatenstelsel,X,Y,Naam,Parameter_Wat_Omschrijving,Compartiment.Code,Compartiment.Omschrijving,Eenheid.Code,Eenheid.Omschrijving,Grootheid.Code,Grootheid.Omschrijving,Hoedanigheid.Code,Hoedanigheid.Omschrijving,Parameter.Code,Parameter.Omschrijving,start,end
0,KEET,18022,25831,566845.068456,5.717765e+06,Keeten boei,Significante golfhoogte in het spectrale domei...,OW,Oppervlaktewater,cm,centimeter,Hm0,Significante golfhoogte in het spectrale domein,F003-050,golffrequentie tussen 30 en 500 mHz,NVT,Waarde is niet van toepassing,1800-01-01,2100-12-31


In [30]:
CHUNK = 8

In [31]:
starts = pd.date_range(start, end, freq = f'{CHUNK}MS').to_pydatetime()

In [33]:
ends = starts + dateutil.relativedelta.relativedelta(months = CHUNK, days = -1)

In [16]:
def range_from_boolean():
    session = requests.session()
    range = []
    for (strt, nd) in list(zip(starts, ends)):
        data_present = com.assert_data_available(sel, strt, nd, session)
        if data_present:
            range.append(list(zip([strt], [nd])))
    return range

In [35]:
range_from_boolean()

[[(datetime.datetime(2020, 9, 1, 0, 0), datetime.datetime(2021, 4, 30, 0, 0))],
 [(datetime.datetime(2021, 5, 1, 0, 0),
   datetime.datetime(2021, 12, 31, 0, 0))],
 [(datetime.datetime(2022, 1, 1, 0, 0), datetime.datetime(2022, 8, 31, 0, 0))]]